In [ ]:
import numpy as np


In [ ]:
import pickle
with open("/kaggle/input/ecodedimg/encoded-img.pkl", "rb") as pickle_in:
    train_features = pickle.load(pickle_in)
with open("/kaggle/input/30kdata/caption-img.pkl", "rb") as pickle_in:
    captions = pickle.load(pickle_in)
with open("/kaggle/input/30kdata/vocab.pkl", "rb") as pickle_in:
    vocab = pickle.load(pickle_in)
print("Vocab : " + str(len(vocab)))
print("train_features : " + str(len(train_features)))
print(captions['1000523639.jpg'])

In [ ]:
#list of all training captions
all_train_captions = []
for key, val in captions.items():
    for caption in val:
        all_train_captions.append(caption)
len(all_train_captions)

In [ ]:
#find the maximum length of a description in a dataset
max_length = max(len(des.split()) for des in all_train_captions)
max_length

In [ ]:
#word mapping to integers
ixtoword = {}
wordtoix = {}

ix = 1
for word in vocab:
    wordtoix[word] = ix
    ixtoword[ix] = word
    ix += 1
len(wordtoix)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
vocab_size = len(ixtoword) + 1
vocab_size

In [ ]:
from keras.models import load_model
model = load_model("/kaggle/working/new-model.h5")

In [ ]:
len(captions)

In [ ]:
epochs = 20
number_pics_per_bath = 128
steps = len(captions)//number_pics_per_bath

In [ ]:
from numpy import array

In [ ]:
import pickle
with open("wordtoix.pkl", "wb") as pickle_out:
    pickle.dump(wordtoix, pickle_out)
with open("ixtoword.pkl", "wb") as pickle_out:
    pickle.dump(ixtoword, pickle_out)

In [ ]:
for i in range(epochs):
    generator = data_generator(captions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    if (i % 3 == 0) :
        model.save('image-caption-30k-' + str(i) + '.h5')

In [ ]:
model.save("new-model-1.h5")
#2.7085